<a href="https://colab.research.google.com/github/Marilyn-S/Machine-Learning-Final/blob/main/MLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>